[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NeuralConceptDev/examples/blob/master/bike.ipynb)

# Introducing the Neural Concept API

Neural Concept provides APIs for training 3D deep learning models that learn to predict engineering simulations of different physical processes. In this tutorial, we are going to show how one of our pre-trained models can be used for making predictions on the 3D model of a bike. At the end of the tutorial, you will learn to -

* explore the public datasets on our platform
* use the public pre-trained models to perform predictions
* learn about the more advanced features of our platform.

To get started, visit https://prod.neuralconcept.com/register and create an account.

## Setup

In this section, we install the required packages and setup our credentials to use the client.

In [ ]:
# Install the ncapi client
!pip install -U ncapi-client
# Install plotly for visualizations
! pip install plotly

In [ ]:
import os

os.environ["NCAPI_URL"] = "https://prod.neuralconcept.com"
os.environ["NCAPI_USERNAME"] = "<INSERT USERNAME>"
os.environ["NCAPI_PASSWORD"] = "<INSERT PASSWORD>"

from ncapi_client.client import Client

client = Client()

## The public bike Dataset

Lets list the available datasets.

In [ ]:
client.datasets

In [ ]:
All users of our platform have access to a dataset of bike designs. <Insert history>.

Lets get some more details about the dataset.

In [ ]:
from ncapi_client.dataset import Dataset
bike_dataset = Dataset(client, "bike-demo")
bike_dataset.info

The dataset info provides detailed information about the dataset. The "data_format" attribute indicates the current format of the dataset, and the formats are the list of formats available for the dataset. The "raw" format is the format that user is expected to upload the data in, we will get into the details of the format in a more advanced notebook. The "npy_indiv" format is a serialized numpy array format, which is ready to be fed into a deep learning model. The status of "CONVERTED" indicates that this dataset has been converted from "raw" to "npy_indiv" format.

Now let us look into some samples present in the dataset. The samples attributes returns the list of sample ids the dataset contains.

In [ ]:
bike_dataset.samples

In [ ]:
sample = bike_dataset.sample("0000")

In [ ]:
sample.keys()

The "verts", "adj" and "faces" attributes describe the 3D shape. The "output_fields" and "output_scalars" are the labels for the sample ( i.e. the variable which the model tries to predict) . Some datasets can also have a "input_fields" and "input_scalars" attribute, which represent inputs to the model in addition to the shape information.

To get a better feel for the problem, lets visualize this sample.

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np

def visualize_sample(sample, predictions=None):
    verts = np.array(sample["verts"])
    faces = np.array(sample["faces"])
    preds = predictions["output_fields"][0, ...] if predictions else np.array(sample["output_fields"])
    x, y, z = verts[:,0], verts[:,1], verts[:,2]
    I, J, K = faces[:,0], faces[:,1], faces[:,2]
    preds = np.clip(preds, -0.5, 0.5)
    trace = go.Mesh3d(x=x,y=y,z=z,i=I,j=J,k=K,intensity=preds[:,0])
    return go.Figure(data=[trace], layout=go.Layout(scene=dict(aspectmode="data")))

In [ ]:
visualize_sample(sample).show()

The visualization shows the 3D model of the bike. The output_field values are shown in the color scale.

## Public Trained Models

Next lets look at the available pre-trained model that can be used to get predictions on our bike shapes.

In [ ]:
client.trained_models

In [ ]:
from ncapi_client.trained_model import TrainedModel
bike_model = TrainedModel(client, "bike-demo-model.ckpt-103600")
bike_model.info

The checkpoint_id attribute is checkpoint step in the training process from which this trained model was created. The class_name is the underlying python class which encapsulates this model and defines its network architecture. The config parameters are additional attributes that define the model for a specific dataset.

## Submitting a prediction Job and get predictions

In this section, we will see how to submit a prediction job using the bike dataset and model we saw in the previous sections. 

To submit a prediction job, we need the trained model id, the dataset id and a list of samples for which we want the predictions. Lets submit the first 10 samples of the dataset as an example.

In [ ]:
from ncapi_client.prediction import Prediction

pred = Prediction.submit(client, bike_model.info.uuid, 
                         dataset_id=bike_dataset.info.uuid, 
                         sample_ids=bike_dataset.samples[:10])

In [ ]:
pred.info

A prediction job has now been created. Behind the scenes, the API will spin up a prediction worker, pull the dataset samples and the model and calculate the predictions. This may take upto 10-15 minutes, as in some cases it may involve provisioning a new virtual machine with GPUs installed. When the job has finished, the status of te Job will change to finished.  

In [ ]:
while pred.info.status != "FINISHED":
    print(".", end="")
print("Job finished.")

In [ ]:
pred.info

We can now fetch the predictions and compare with the ground truth labels. To get the predictions, we pass in a list of sample ids. This may take a while, as the predictions are fetched one by one. We are coming up soon with a batch API.

In [ ]:
results = pred.get_results(bike_dataset.samples[:10])

We can now visualize the predictions for the first sample and compare with the ground truth.

In [ ]:
visualize_sample(results[0].sample, results[0].prediction).show()

In [ ]:
visualize_sample(sample).show()

## Running an interactive session

In addition to submitting a prediction job, you can also create a prediction endpoint to get realtime predictions. You can use this feature to interatively make changes to your design and get immediate feedback in the form of how your changes affect the prediction.

To start an interactive session, you need a trained model and a dataset sample.

In [ ]:
from ncapi_client.session import Session

sess = Session.start(client, bike_model.info.uuid, bike_dataset.info.uuid, bike_dataset.samples[0], socket_params=dict(max_size=2**24))

Similar to batch prediction, an interactive session may also involve provisioning a new instance and may take upto 10-15 minutes.

In [ ]:
sess.info

The interactive session can be viewed in the following url -

In [ ]:
f"https://staging.neuralconcept.com/viewer/session/{sess.info.id}?jwt={client.access_token}"

## Delete Resources

In [ ]:
sess.delete()
pred.delete()

## Summary

In this notebook, we showcased how the Neural Concept API can be used for getting batch and interactive predictions on 3D models. 
In a follow up example, we will show how you can bring your data and create your own trained models and use them for predictions.
